**Verifying Assignment 2 S3 Bucket Creation**

In [7]:
import boto3
import sagemaker

# Checking that bucket has been created successfully
session = boto3.session.Session()
region = session.region_name
sagemaker_session = sagemaker.Session()
bucket = "spaz-aai504-assignment2-bucket"

s3 = boto3.Session().client(service_name="s3", region_name=region)

In [8]:
from botocore.client import ClientError

response = None

try:
    response = s3.head_bucket(Bucket=bucket)
    print(response)
    setup_s3_bucket_passed = True
except ClientError as e:
    print("[ERROR] Cannot find bucket {} in {} due to {}.".format(bucket, response, e))

{'ResponseMetadata': {'RequestId': 'XGNANP7GJ1AS0FWS', 'HostId': 'xc51Uu8Mc1uH/06UocHmQJLt/4N/gXVdGs/dx2gYU66pahGULwYZMFiMfqjJd8A7peH5oTfBBtpy52NMgenqfhD+twaysP23gUMbM+RpbVk=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'xc51Uu8Mc1uH/06UocHmQJLt/4N/gXVdGs/dx2gYU66pahGULwYZMFiMfqjJd8A7peH5oTfBBtpy52NMgenqfhD+twaysP23gUMbM+RpbVk=', 'x-amz-request-id': 'XGNANP7GJ1AS0FWS', 'date': 'Tue, 16 Sep 2025 02:17:18 GMT', 'x-amz-bucket-region': 'us-east-1', 'x-amz-access-point-alias': 'false', 'content-type': 'application/xml', 'transfer-encoding': 'chunked', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'BucketRegion': 'us-east-1', 'AccessPointAlias': False}


In [9]:
# Listing first 10 objects to confirm data presence
response = s3.list_objects_v2(Bucket=bucket, Prefix="", MaxKeys=10)
if "Contents" in response:
    print("Found data files in bucket:")
    for obj in response["Contents"]:
        print(f" - {obj['Key']} ({obj['Size']} bytes)")
else:
    print("No objects found under that prefix.")

Found data files in bucket:
 - dataset.csv (20118244 bytes)


**Creating Athena Database**

In [10]:
from pyathena import connect

database_name = "aai540_assignment2_db"

In [11]:
# Setting S3 staging directory
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

In [12]:
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [13]:
statement = "CREATE DATABASE IF NOT EXISTS {}".format(database_name)
print(statement)

CREATE DATABASE IF NOT EXISTS aai540_assignment2_db


In [15]:
import pandas as pd

pd.read_sql(statement, conn)

/tmp/ipykernel_4655/3803073958.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


""


In [16]:
# Verifying database has been created successfully
statement = "SHOW DATABASES"

df_show = pd.read_sql(statement, conn)
df_show.head(5)

/tmp/ipykernel_4655/328402195.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(statement, conn)


,database_name
0,aai540_assignment2_db
1,default
2,dsoaws


In [17]:
# Setting Athena parameters
table_name = "aai540_assignment2_table"

In [41]:
# Creating Athena table
s3_private_path_assignment2 = "s3://{}/aai_assignment2/dataset"
statement = f"""CREATE EXTERNAL TABLE IF NOT EXISTS {database_name}.{table_name}(
    unnamed__0 int,
    track_id string,
    artists string,
    album_name string,
    track_name string,
    popularity int,
    duration_ms bigint,
    explicit boolean,
    danceability double,
    energy double,
    key int,
    loudness double,
    mode int,
    speechiness double,
    acousticness double,
    instrumentalness double,
    liveness double,
    valence double,
    tempo double,
    time_signature int,
    track_genre string
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe'
WITH SERDEPROPERTIES (
    'serialization.format' = ',',
    'field.delim' = ','
)
LOCATION 's3://{bucket}/'
TBLPROPERTIES (
    'skip.header.line.count'='1'
)
"""

print(statement)

CREATE EXTERNAL TABLE IF NOT EXISTS aai540_assignment2_db.aai540_assignment2_table(
    unnamed__0 int,
    track_id string,
    artists string,
    album_name string,
    track_name string,
    popularity int,
    duration_ms bigint,
    explicit boolean,
    danceability double,
    energy double,
    key int,
    loudness double,
    mode int,
    speechiness double,
    acousticness double,
    instrumentalness double,
    liveness double,
    valence double,
    tempo double,
    time_signature int,
    track_genre string
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe'
WITH SERDEPROPERTIES (
    'serialization.format' = ',',
    'field.delim' = ','
)
LOCATION 's3://spaz-aai504-assignment2-bucket/'
TBLPROPERTIES (
    'skip.header.line.count'='1'
)



In [42]:
# Creating table in Athena
pd.read_sql(statement, conn)

/tmp/ipykernel_4655/3728654170.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


""


In [37]:
statement = "SHOW TABLES in {}".format(database_name)

df_show = pd.read_sql(statement, conn)
df_show.head(5)

/tmp/ipykernel_4655/2201015668.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(statement, conn)


,tab_name
0,aai540_assignment2_table


In [43]:
# Running Sample SQL Query
statement = """
SELECT
track_name,
energy
FROM {}.{}
WHERE energy >= 0.5
""".format(database_name, table_name)
df = pd.read_sql(statement, conn)
print(df)

/tmp/ipykernel_4655/1306474280.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(statement, conn)


                             track_name  energy
0                                Hunger   0.632
1                       Hold On - Remix   0.780
2      Falling in Love at a Coffee Shop   0.561
3                               Vol. 4"   0.717
4                               Vol. 3"   0.678
...                                 ...     ...
81862        Stay (You Are Good) - Live   0.762
81863       At The Cross (Love Ran Red)   0.531
81864             Your Love Never Fails   0.860
81865       How Can I Keep From Singing   0.687
81866                           Friends   0.506

[81867 rows x 2 columns]


**Running Queries with SQL and Pandas**

In [60]:
# 1. List artist, track_name, and popularity for songs that have a popularity greater than or equal to 99
query_1 = """
SELECT artists, track_name, popularity
FROM {}.{}
WHERE popularity >= 99
""".format(database_name, table_name)

df = pd.read_sql(query_1, conn)
print(df)

/tmp/ipykernel_4655/521511105.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query_1, conn)


                artists                 track_name  popularity
0  Sam Smith;Kim Petras  Unholy (feat. Kim Petras)         100
1     Charlie Brown Jr.               Prazo Longo"         333
2          Smyang Piano                    Vol. 4"      134340
3  Sam Smith;Kim Petras  Unholy (feat. Kim Petras)         100


In [61]:
# 2. List artists with an average popularity of 92
query_2 = """
SELECT artists, AVG(popularity) AS avg_popularity
FROM {}.{}
GROUP BY artists
HAVING AVG(popularity) = 92
""".format(database_name, table_name)

df = pd.read_sql(query_2, conn)
print(df)

/tmp/ipykernel_4655/3056184965.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query_2, conn)


             artists  avg_popularity
0       Harry Styles            92.0
1  Rema;Selena Gomez            92.0


In [62]:
#3. List the Top 10 genres with the highest average energy
query_3 = """
SELECT track_genre, AVG(energy) AS avg_energy
FROM {}.{}
GROUP BY track_genre
ORDER BY avg_energy DESC
LIMIT 10
""".format(database_name, table_name)

df = pd.read_sql(query_3, conn)
print(df)

/tmp/ipykernel_4655/3702353173.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query_3, conn)


  track_genre  avg_energy
0       0.797   1174026.0
1       0.556    691306.0
2      0.0371    629420.0
3      0.0359    614791.0
4       0.492    542000.0
5        0.45    538160.0
6       0.914    531293.0
7      0.0427    526946.0
8      0.0761    502786.0
9      0.0346    500088.0


In [64]:
#4. How many tracks is Bad Bunny on?
query_4 = """
SELECT COUNT(*) AS bad_bunny_tracks
FROM {}.{}
WHERE LOWER(artists) LIKE '%bad bunny%'
""".format(database_name, table_name)

df = pd.read_sql(query_4, conn)
print(df)

/tmp/ipykernel_4655/730192103.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query_4, conn)


   bad_bunny_tracks
0               416


In [65]:
#5. Show the top 10 genres in terms of popularity, sorted by their most popular track
query_5 = """
SELECT track_genre, MAX(popularity) AS max_popularity
FROM {}.{}
GROUP BY track_genre
ORDER BY max_popularity DESC
LIMIT 10
""".format(database_name, table_name)

df = pd.read_sql(query_5, conn)
print(df)

/tmp/ipykernel_4655/1320050212.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query_5, conn)


  track_genre  max_popularity
0           4          134340
1       dance             100
2         pop             100
3      latino              98
4      reggae              98
5         edm              98
6       latin              98
7   reggaeton              98
8        rock              96
9       piano              96


In [66]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

*** SIGTERM received at time=1757999243 on cpu 1 ***
PC: @     0x7fd871fbbe2e  (unknown)  epoll_wait
    @     0x7fd812d4eb0d         64  absl::lts_20240722::AbslFailureSignalHandler()
    @     0x7fd871ed8520  (unknown)  (unknown)
[2025-09-16 05:07:23,523 E 4655 4655] logging.cc:497: *** SIGTERM received at time=1757999243 on cpu 1 ***
[2025-09-16 05:07:23,523 E 4655 4655] logging.cc:497: PC: @     0x7fd871fbbe2e  (unknown)  epoll_wait
[2025-09-16 05:07:23,524 E 4655 4655] logging.cc:497:     @     0x7fd812d4eb39         64  absl::lts_20240722::AbslFailureSignalHandler()
[2025-09-16 05:07:23,524 E 4655 4655] logging.cc:497:     @     0x7fd871ed8520  (unknown)  (unknown)
